# Vision Constants Optimization

This notebook is made to optimize vision constants.

This is done by the following steps:
1. Define vision calculation functiosn - exactly the same to what are used on the bot. Check that the results produced are the same.
2. Record true values to the target and corresponding camera pixels as reference for optimization.
3. Run the optimization.
4. Test if the result is accurate.

In [1]:
# Define vision functions.
# All angles in radians, all distance in meters.
import math

focalLength = 751.2954296909647
targetHeight = 1.70


def calculateDistanceToTargetMeters(cameraHeightMeters, targetHeightMeters, cameraPitchRadians, targetPitchRadians):
    return (targetHeightMeters - cameraHeightMeters) / math.tan(cameraPitchRadians + targetPitchRadians)


def estimateCameraToTargetTranslation(targetDistanceMeters, yaw):
    return [math.cos(yaw) * targetDistanceMeters, math.sin(yaw) * targetDistanceMeters]


def calculatePitch(offset, target, focal):
    return -math.degrees(math.atan((offset - target) / focal))


def calculateYaw(offset, target, focal):
    return math.degrees(math.atan((offset - target) / focal))


def solveCameraToGoalTranslation(corner, goalHeight, lenHeight, pitchAngle):
    x = corner[0]
    y = corner[1]

    pitch = math.radians(calculatePitch(y, 480/2.0, focalLength))
    yaw = math.radians(calculateYaw(x, 640 / 2.0, focalLength))

    rangeLength = calculateDistanceToTargetMeters(
        lenHeight, goalHeight, pitchAngle, pitch)
    return estimateCameraToTargetTranslation(rangeLength, yaw)


def visionFunction(corner, len_height, horizontal):
    return solveCameraToGoalTranslation(corner, targetHeight, len_height, horizontal)


In [2]:
# Test Vision Function
# The bot at the time shows estimated distance to be around 2.2912, the same with what the vision functions outputs
visionFunction([356.5,401], 1.53, math.radians(30))

[0.5255642226726529, 0.025533356612381545]

In [3]:
# Recording all the measured true corresponding values as a reference for optimization
# [distance x, distance y, pixel x, pixel y]
true_values = [
    [1.0, 0.35, 413.5, 330.5],
    [1.1, 0.35, 417, 342.5],
    [1.2, 0.35, 418.5, 352.5],
    [1.3, 0.35, 411.5, 360.5],
    [1.4, 0.35, 414.5, 368.5],
    [1.5, 0.36, 408.5, 375.5],
    [1.6, 0.36, 407.5, 381.5],
    [1.7, 0.36, 399, 387.5],
    [1.8, 0.37, 401.5, 393.5],
    [1.9, 0.38, 405, 399.5],
    [2.0, 0.40, 408.5, 404],
    [2.1, 0.0, 250, 100],
    [2.2, 0.0, 250, 100],
    [2.3, 0.0, 250, 100],
    [2.4, 0.0, 250, 100],
    [2.5, 0.0, 250, 100],
    [2.6, 0.0, 250, 100],
    [2.7, 0.0, 250, 100],
    [2.8, 0.0, 250, 100],
    [2.9, 0.0, 250, 100],
    [3.0, 0.0, 250, 100],
    [3.1, 0.0, 250, 100],
    [3.2, 0.0, 250, 100],
    [3.3, 0.0, 250, 100],
    [3.4, 0.0, 250, 100],
    [3.5, 0.0, 250, 100]
]


In [4]:
# Define error function as the target function for optimization
def targetErrorFunction(params):
    len_height = params[0]
    horizontal = params[1]
    
    error_sum = 0.0
    for value in true_values:
        estimate = visionFunction([value[2], value[3]], len_height, horizontal)
        # Simple variance
        error = math.sqrt((estimate[0] - value[0])**2 + (estimate[1] - value[1])**2)
        error_sum += error
    return error_sum

In [5]:
# Test error function
targetErrorFunction([1,2])

71.2237937302535

In [8]:
import scipy.optimize as optimize

# Height, Angle To Horizontal
initial_guess = [1.53, math.radians(45.0)]
bounds = [(0.0, 2.0), (math.radians(0.0), math.radians(90.0))]
print("Before Optimization: Height %.2f m, Angle %.2f deg" % (initial_guess[0], math.degrees(initial_guess[1])))

# Run Optimization
result = optimize.minimize(targetErrorFunction, initial_guess, bounds=bounds)
if result.success:
    fitted_params = result.x
    print("After Optimization: Height %.2f m, Angle %.2f deg" % (fitted_params[0], math.degrees(fitted_params[1])))
else:
    raise ValueError(result.message)

Before Optimization: Height 1.53 m, Angle 45.00 deg
After Optimization: Height 0.54 m, Angle 17.04 deg
